In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
import json, math
import matplotlib.pyplot as plt
from folium import plugins
import re

from matplotlib import font_manager, rc, rcParams
# def set_korea_font():
#     font_name = font_manager.FontProperties(fname="/System/Library/Fonts/Supplemental/AppleGothic.ttf").get_name()
#     rc('font', family=font_name)
#     rcParams.update({'font.size': 11})
#     rcParams['axes.unicode_minus'] = False  
# set_korea_font()

from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi

from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
import datetime


In [2]:
os.chdir('C:\\Users\\simjh96\\OneDrive\\문서\\Car_Accident\\Feature\\data')

In [3]:
#tab에서 찾기 편하게 모든 단어에 _count_ 붙음
#refer_data 는 data_set['function name']
#홍익선
overspeed_cam_count_ = pd.read_csv('overspeed_cam.csv')
floating_pop_count_ = pd.read_csv('floating_pop.csv')
child_pedestrian_count_ = pd.read_csv('child_pedestrian.csv')
bump_count_ = pd.read_csv('bump.csv')
parking_count_ = pd.read_csv('parking.csv')
parking_cctv_count_ = [parking_count_, pd.read_csv('parking_cctv.csv')]

#조민주
car_count_ = gpd.read_file("3.오산시_차량등록현황_격자.geojson", encoding = "utf-8")
child_count_ = gpd.read_file("child_count.geojson", encoding = "utf-8")
child_rate_count_ = gpd.read_file("child_rate.geojson", encoding = "utf-8")
elem_kinder_count_ = gpd.read_file("elem_kinder_count.csv", encoding = "utf-8")
isSchoolZone_count_ = pd.read_csv("9.오산시_어린이보호구역(re).csv", encoding = "utf-8")
numberSchoolZone_count_ = isSchoolZone_count_

#심재훈
accident_count_ = pd.read_csv("accident_count.csv")
chaos1_nearby_count_ = [gpd.read_file('23.오산시_상세도로망_LV6.geojson'),
                 pd.read_csv('chaos1.csv')]
width_nearby_count_ = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
cross_road_nearby_count_ = width_nearby_count_

#이재훈
num_cram_school_count_ = pd.read_csv("학원.csv")
shortest_cross_count_ = gpd.read_file("횡단보도.json",encoding="utf-8")
shortest_traffic_signal_count_ = gpd.read_file('19.오산시_신호등.geojson',encoding="utf-8")
shortest_sidewalk_count_ =  gpd.read_file("인도.json",encoding="utf-8")


data_set = {
'overspeed_cam_count': overspeed_cam_count_,
'floating_pop_count':floating_pop_count_,
'child_pedestrian_count':child_pedestrian_count_,
'bump_count':bump_count_,
'parking_count':parking_count_,
'parking_cctv_count':parking_cctv_count_,
'car_count':car_count_,
'child_count':child_count_,
'child_rate_count':child_rate_count_,
'elem_kinder_count':elem_kinder_count_,
'isSchoolZone_count':isSchoolZone_count_,
'numberSchoolZone_count':numberSchoolZone_count_,
'accident_count':accident_count_,
'chaos1_nearby_count':chaos1_nearby_count_,
'width_nearby_count':width_nearby_count_,
'cross_road_nearby_count':cross_road_nearby_count_,
'num_cram_school_count':num_cram_school_count_,
'shortest_cross_count':shortest_cross_count_,
'shortest_traffic_signal_count':shortest_traffic_signal_count_,
'shortest_sidewalk_count':shortest_sidewalk_count_,
}

In [4]:
os.chdir('C:\\Users\\simjh96\\OneDrive\\문서\\Car_Accident\\data\\speed')

In [32]:
speeds = pd.read_excel('2018년_평균속도_경기도 오산시_주요도로망Level5.5_도로단위_중복제거2.xlsx')
speeds.road_name = speeds.road_name.apply(str)
speeds

,level5.5_linkID,ITS_linkID,road_rank,road_length,road_name,sido_code,sigungu_code,emd_code,sido_name,sigungu_name,emd_name,week_type,velocity_AVRG
0,1079787,2140611600,107,1.129,엘지로,31,31140,3114052,경기도,오산시,대원동,weekend,37.08
1,1079788,2230000841,106,0.506,남부대로,31,31140,3114052,경기도,오산시,대원동,weekend,33.73
2,1079789,2230069301,107,0.930,동부대로,31,31140,3114052,경기도,오산시,대원동,weekend,39.42
3,1079790,2140188000,107,0.170,동부대로,31,31140,3114052,경기도,오산시,대원동,weekend,31.68
4,1079791,2140609700,107,1.129,엘지로,31,31140,3114052,경기도,오산시,대원동,weekend,36.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,1063338,2230019800,107,0.017,서동로,31,31140,3114056,경기도,오산시,초평동,weekend,22.67
421,1063339,0,107,0.015,가장로,31,31140,3114056,경기도,오산시,초평동,weekend,11.17
422,1063340,2230019800,107,0.009,서동로,31,31140,3114056,경기도,오산시,초평동,weekend,16.26
423,1063341,2230038900,107,0.019,서동로,31,31140,3114056,경기도,오산시,초평동,weekend,24.54


In [17]:
compare = data_set['width_nearby_count'].loc[data_set['width_nearby_count'].road_rank == '107']

In [31]:
compare.road_name = compare.road_name.apply(str)

In [50]:
set_s = set(speeds.road_name)
# set(compare.road_name)

In [47]:
speeds.groupby('road_name').apply(lambda x: x.count())

,level5.5_linkID,ITS_linkID,road_rank,road_length,road_name,sido_code,sigungu_code,emd_code,sido_name,sigungu_name,emd_name,week_type,velocity_AVRG
road_name,,,,,,,,,,,,,
nan,5,5,5,5,5,5,5,5,5,5,5,5,5
가장로,20,20,20,20,20,20,20,20,20,20,20,20,20
경기대로,43,43,43,43,43,43,43,43,43,43,43,43,43
경기대로761번길,3,3,3,3,3,3,3,3,3,3,3,3,3
경기동로,11,11,11,11,11,11,11,11,11,11,11,11,11
경부고속도로,10,10,10,10,10,10,10,10,10,10,10,10,10
궐리사로,6,6,6,6,6,6,6,6,6,6,6,6,6
남부대로,16,16,16,16,16,16,16,16,16,16,16,16,16
내삼미로,4,4,4,4,4,4,4,4,4,4,4,4,4


In [60]:
count_c = compare.groupby('road_name').apply(lambda x: x.count()).iloc[:,[0]]
count_s = speeds.groupby('road_name').apply(lambda x: x.count()).iloc[:,[0]]

In [63]:
count_c

,link_id
road_name,
None,293
가수로,5
가장로,13
가장산업동로,17
가장산업북로,3
...,...
한신대길,3
현충로,13
현충로72번길,3


In [66]:
pd.merge(count_c, count_s, 'left', on = 'road_name').iloc[0:20,:]

,link_id,level5.5_linkID
road_name,,
None,293,NaN
가수로,5,NaN
가장로,13,20.0
가장산업동로,17,NaN
가장산업북로,3,NaN
가장산업서로,18,NaN
가장산업서북로,22,NaN
강변로,1,NaN
경기대로,46,43.0


In [68]:
speeds[speeds.road_name == '경기대로761번길']

,level5.5_linkID,ITS_linkID,road_rank,road_length,road_name,sido_code,sigungu_code,emd_code,sido_name,sigungu_name,emd_name,week_type,velocity_AVRG
206,5103023,2230180400,107,0.560,경기대로761번길,31,31140,3114055,경기도,오산시,세마동,weekend,37.69
207,5103024,2230180500,107,0.560,경기대로761번길,31,31140,3114055,경기도,오산시,세마동,weekend,31.37
208,5103025,2230180500,107,0.027,경기대로761번길,31,31140,3114055,경기도,오산시,세마동,weekend,30.94


In [67]:
compare[compare.road_name == '경기대로761번길']

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,facil_name,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,geometry
1207,571350715,60,경기대로761번길,0,107,32768,0,None,3,2,0,0.137,4,0,0,0,"MULTILINESTRING ((127.04466 37.18785, 127.0447..."
1348,571353616,60,경기대로761번길,0,107,32768,0,None,3,2,0,0.16,4,0,0,0,"MULTILINESTRING ((127.04399 37.18919, 127.0446..."
1802,571369622,60,경기대로761번길,0,107,4,0,None,1,0,1,0.027,1,0,0,0,"MULTILINESTRING ((127.04742 37.18610, 127.0472..."
1851,571372097,60,경기대로761번길,0,107,32768,0,None,3,2,0,0.074,4,0,0,0,"MULTILINESTRING ((127.04524 37.18670, 127.0455..."
1852,571372099,60,경기대로761번길,0,107,32768,0,None,3,2,0,0.076,4,0,0,0,"MULTILINESTRING ((127.04726 37.18588, 127.0467..."
1853,571372100,60,경기대로761번길,0,107,32768,0,None,3,2,0,0.113,4,0,0,0,"MULTILINESTRING ((127.04556 37.18608, 127.0457..."


In [52]:
set_c = set(compare.road_name)
set_c

{'None',
 '가수로',
 '가장로',
 '가장산업동로',
 '가장산업북로',
 '가장산업서로',
 '가장산업서북로',
 '강변로',
 '경기대로',
 '경기대로148번길',
 '경기대로25번길',
 '경기대로658번길',
 '경기대로761번길',
 '경기동로',
 '경기동로186번길',
 '경기동로8번길',
 '궐동로',
 '궐리사로',
 '궐리사로29번길',
 '금암로',
 '금암로11번길',
 '금암로16번길',
 '남부대로',
 '내삼미로',
 '내삼미로48번길',
 '내삼미로79번길',
 '내삼미로80번길',
 '누읍로',
 '누읍로46번길',
 '대원로',
 '대호로',
 '대호로117번길',
 '대호로71번길',
 '독산성로',
 '독산성로232번길',
 '독산성로270번길',
 '독산성로310번길',
 '독산성로449번길',
 '동부대로',
 '동부대로436번길',
 '매홀로',
 '문시로',
 '문헌공로',
 '밀머리로1번길',
 '박동길',
 '발안로',
 '배무니로',
 '법원로',
 '부원로',
 '북삼미로',
 '삼미로',
 '삼미로47번길',
 '서동로',
 '서부로',
 '성호대로',
 '성호대로40번길',
 '성호대로50번길',
 '세남로',
 '세남로14번길',
 '세마문학로',
 '세마역로',
 '세마역로19번길',
 '세마역로41번길',
 '수목원로',
 '수목원로468번길',
 '수목원로49번길',
 '수목원로88번길',
 '수청로',
 '수청로50번길',
 '수청로8번길',
 '양산로',
 '양산로398번길',
 '양산로410번길',
 '엘지로',
 '여계산로',
 '역광장로',
 '오산대역로',
 '오산대역로132번길',
 '오산로',
 '오산로132번길',
 '오산로160번길',
 '오산로190번길',
 '오산로272번길',
 '오산로290번길',
 '오산로360번길',
 '오산로50번길',
 '오산로62번길',
 '오산천로',
 '오산천로158번길',
 '외삼미로',
 '외삼미로15번길',
 '외삼미로5번길',


In [14]:
speeds.sort_values('road_length').road_length

416    0.005
77     0.009
422    0.009
81     0.009
419    0.009
       ...  
226    2.201
358    3.111
254    3.636
247    5.012
246    5.321
Name: road_length, Length: 425, dtype: float64

In [13]:
compare.sort_values('length').length

950     0.000046
1552    0.000041
385     0.000047
696     0.000054
1708    0.000060
          ...   
996     0.006187
1570    0.006428
1630    0.006678
1202    0.007746
231     0.015675
Length: 1541, dtype: float64

In [22]:
data_set['width_nearby_count'].length.loc[data_set['width_nearby_count'].length*1000 > 5.321]

1       0.005914
46      0.007480
47      0.005919
122     0.006898
168     0.006254
184     0.005665
231     0.015675
402     0.006032
473     0.005349
555     0.006356
588     0.005975
670     0.007429
767     0.011327
818     0.009370
996     0.006187
1044    0.012769
1045    0.011883
1200    0.005371
1202    0.007746
1217    0.005761
1220    0.006591
1233    0.008839
1234    0.011706
1239    0.010355
1240    0.011821
1242    0.034823
1248    0.034691
1265    0.006705
1505    0.007099
1514    0.006312
1570    0.006428
1590    0.005803
1630    0.006678
1676    0.006103
1867    0.007304
1868    0.006794
1870    0.007032
1871    0.010322
dtype: float64

In [35]:
data_set['width_nearby_count']

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,facil_name,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,geometry
0,478344684,60,수도권제2순환고속도로(봉담동탄,400,108,2,0,None,1,0,1,0.472,1,0,0,0,"MULTILINESTRING ((126.99779 37.17385, 126.9979..."
1,478344686,60,수도권제2순환고속도로(봉담동탄,400,108,2,0,None,2,0,1,0.606,2,0,0,0,"MULTILINESTRING ((126.99779 37.17385, 126.9978..."
2,478344690,60,수도권제2순환고속도로(봉담동탄,400,108,2,0,None,1,0,1,0.383,1,0,0,0,"MULTILINESTRING ((127.00104 37.18002, 127.0009..."
3,478344696,60,오산화성고속도로,171,108,2,0,None,1,0,1,0.488,1,0,0,0,"MULTILINESTRING ((126.99614 37.17621, 126.9963..."
4,478344700,40,오산화성고속도로,171,108,2,0,None,1,0,1,0.28,1,0,0,0,"MULTILINESTRING ((126.99770 37.17692, 126.9975..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,571376981,100,수도권제2순환고속도로(봉담동탄,400,101,1,0,None,2,0,1,0.643,2,1,0,0,"MULTILINESTRING ((127.04011 37.17816, 127.0467..."
1871,571376982,100,수도권제2순환고속도로(봉담동탄,400,101,1,0,None,2,0,1,0.936,2,1,0,0,"MULTILINESTRING ((127.04770 37.18087, 127.0478..."
1872,571376983,100,수도권제2순환고속도로(봉담동탄,400,101,1,2,세교지하차도,2,0,1,0.093,2,1,0,0,"MULTILINESTRING ((127.04674 37.18053, 127.0477..."
1873,900000166,0,None,0,107,32768,0,None,1,0,1,0.065,1,0,0,0,"MULTILINESTRING ((127.05992 37.16942, 127.0601..."
